In [17]:
# It defines variables x_1,...,x_n
def variables(n):
    return [var('x_%s'%k) for k in range(1,n+1)]

# Given a polytope P = [F1,...,Fm] as a list of facets where Fi is a list with the vertices in that facet,
# it returns the number of variables that appear in the symbolic slack matrix
def num_variables(P):
    num_facets = len(P)
    num_vertices = max([max(F) for F in P])
    num_zeros = sum([len(F) for F in P])
    return num_facets*num_vertices - num_zeros
    
# Given a polytope P = [F1,...,Fm] as a list of facets where Fi is a list with the vertices in that facet,
# it returns the symbolic slack matrix
def symbolic_slack_matrix(P):
    num_facets = len(P)
    num_vertices = max([max(F) for F in P])
    num_zeros = sum([len(F) for F in P])
    x = variables(num_facets*num_vertices - num_zeros)
    S = [[0 for j in range(num_vertices)] for i in range(num_facets)]
    c = 0
    for i in range(num_facets):
        for v in range(1, num_vertices+1):
            if v not in P[i]:
                S[i][v-1] = x[c]
                c = c + 1
    return matrix(S)

# Given a symbolic slack matrix S, it returns the binomials and trinomials that are k-minors of S
def binomials_and_trinomials_minors(S, k):
    B = []
    T = []
    minors = S.minors(k)
    for pol in minors:
        s = str(pol)[1:]
        if s.count('+') + s.count('-') == 1:
            if (pol not in B) or (-pol not in B): B.append(pol)
        if s.count('+') + s.count('-') == 2:
            if (pol not in B) or (-pol not in B): T.append(pol)
    return [B,T]
    
# Given a binomial and the number of variables, it transforms it to the form x^c=1 and returns c
def coefficient_binomial(pol, numvar):
    s = str(pol.expand())
    if s[0] == '-':
        i = s.index('+')
        m1 = s[1:i]
        m2 = s[i+1:]
    else:
        i = s.index('-')
        m1 = s[:i]
        m2 = s[i+1:]
    m1 = m1[:-1]
    m1 += '*'
    m2 += '*'
    c = [0 for k in range(numvar)]
    for k in range(1,numvar+1):
        if 'x_' + str(k) + '*' in m1: c[k-1] += 1
        if 'x_' + str(k) + '*' in m2: c[k-1] -= 1
    return vector(c)

# Given a list of binomials and the number of variables,
# it transforms each one to the form x^c=1 and returns the list of c's
def coefficients_binomials(lpol, numvar):
    L = []
    for pol in lpol:
        L.append(coefficient_binomial(pol, numvar))
    return L
 
# Given a trinomial and the number of variables, it transforms it into the form x^a + 1 = x^b and returns [a,b]
def coefficient_trinomial(pol, numvar):
    s = str(pol.expand())
    a = [0 for k in range(numvar)]
    b = [0 for k in range(numvar)]
    if s[0] == '-':
        if s.count('-') == 1:
            i1 = s.index('+')
            i2 = (i1+1) + s[i1+1:].index('+')
            m1 = s[i1+2:i2-1]
            m2 = s[i2+2:]
            m3 = s[1:i1-1]
        else:
            i1 = s.index('+')
            i2 = 1 + s[1:].index('-')
            if i1 < i2:
                m1 = s[1:i1-1]
                m2 = s[i2+2:]
                m3 = s[i1+2:i2-1]
            else:
                m1 = s[1:i2-1]
                m2 = s[i2+2:i1-1]
                m3 = s[i1+2:]
    else:
        if s.count('-') == 1:
            i1 = s.index('+')
            i2 = s.index('-')
            if i1 < i2:
                m1 = s[:i1-1]
                m2 = s[i1+2:i2-1]
                m3 = s[i2+2:]
            else:
                m1 = s[:i2-1]
                m2 = s[i1+2:]
                m3 = s[i2+2:i1-1]
        else:
            i1 = s.index('-')
            i2 = (i1+1) + s[i1+1:].index('-')
            m1 = s[i1+2:i2-1]
            m2 = s[i2+2:]
            m3 = s[:i1-1]
    m1 += '*'
    m2 += '*'
    m3 += '*'
    for k in range(1,numvar+1):
        if 'x_' + str(k) + '*' in m1: a[k-1] += 1
        if 'x_' + str(k) + '*' in m3: b[k-1] += 1    
        if 'x_' + str(k) + '*' in m2:
            a[k-1] -= 1
            b[k-1] -= 1
    return [vector(a),vector(b)]
            
# Given a list of trinomials and the number of variables, it transforms each one to the form x^a + 1 = x^b and returns
# the list of [a,b]'s            
def coefficients_trinomials(lpol, numvar):
    L = []
    for pol in lpol:
        L.append(coefficient_trinomial(pol, numvar))
    return L

#--------------------------------------------------------------------------------------
# Auxiliary functions for ComplexPsdMinimality1_3d and ComplexPsdMinimality2_3d

# Given the (symbolic) slack matrix, it returns the quadrilateral facets
def four_facets(S):
    num_facets = len(list(S))
    L = []
    for i in range(num_facets):
        if list(S[i]).count(0) == 4: L.append(i+1)
    return L

# Given the (symbolic) slack matrix, it returns the 4-degree vertices
def four_vertices(S):
    return four_facets(S.transpose())

# Given the (symbolic) slack matrix, it returns the list of pairs [F,v] where v is in F, F is a quadrilateral facets,
# and v has degree 4
def four_facets_vertices(S):
    L = []
    for F in four_facets(S):
        for v in four_vertices(S):
            if S[F-1][v-1] == 0: L.append([F,v])
    return L

# Given the (symbolic) slack matrix and a facet F, it returns the list of vertices in that facet
def vertices_in_facet(S, F):
    num_vertices = len(list(S[0]))
    L = []
    for j in range(num_vertices):
        if S[F-1][j] == 0: L.append(j+1)
    return L

# Given the (symbolic) slack matrix and a vertex v, it returns the list of the facets containing v
def facets_containing_vertex(S, v):
    return vertices_in_facet(S.transpose(), v)

# Given the (symbolic) slack matrix and a facet F, it returns the list of facets that share an edge with F
def facets_adjacent_to_facet(S, F):
    num_facets = len(list(S))
    L = []
    for i in range(num_facets):
        if (i != F-1) and (len(set(vertices_in_facet(S,i+1)) & set(vertices_in_facet(S,F))) == 2): L.append(i+1)
    return L

# Given the (symbolic) slack matrix and a vertex v, it returns the list of vertices adjacent to v
def vertices_adjacent_to_vertex(S, v):
    return facets_adjacent_to_facet(S.transpose(), v)

# Given a list [a1,...,an], it returns [a1-1,...,an-1]
def fix_indexing(L0):
    L = list(L0)
    for i in range(len(L0)):
        L[i] = L0[i] - 1
    return L

# Given the (symbolic) slack matrix and a quadrilateral facet F, it returns the binomial associated to that facet 
def binomial_facet(S, F):
    return S[fix_indexing(facets_adjacent_to_facet(S, F)), fix_indexing(vertices_in_facet(S, F))].det()

# Given the (symbolic) slack matrix and a 4-degree vertex v, it returns the binomial associated to that vertex 
def binomial_vertex(S, v):
    return binomial_facet(S.transpose(), v)

# Given the (symbolic) slack matrix, it returns the binomials associated to quadrilateral facets and 4-degree vertices
def binomials_3d(S):
    L = []
    for F in four_facets(S): L.append(binomial_facet(S, F))
    for v in four_vertices(S): L.append(binomial_vertex(S, v))
    return L

# Given the (symbolic) slack matrix, a quadrilateral facet F, and a 4-degree vertex v in F, it returns the trinomials associated
# to (F,v)
def trinomials_facet_vertex(S, F, v):
    L = []
    N1 = facets_containing_vertex(S, v)
    N1.remove(F)
    N2 = vertices_in_facet(S, F)
    N2.remove(v)
    return S[fix_indexing(N1), fix_indexing(N2)].det()

# Given the (symbolic) slack matrix, it returns the list of trinomials associated to 4-degree vertices in quadrilateral facets
def trinomials_3d(S): 
    L = []
    for (F,v) in four_facets_vertices(S): L.append(trinomials_facet_vertex(S, F, v))
    return L

#--------------------------------------------------------------------------------------

# Given a d-polytope P = [F1,...,Fm] as a list of facets where Fi is a list with the vertices in that facet,
# it returns if we can conclude that the polytope is not complex psd-minimal using the first trinomial obstruction
# lattice criterion with the (d+2)-binomials and trinomials minors of the symbolic slack matrix of P  
def ComplexPsdMinimality1(P, d):
    numvar = num_variables(P)
    S = symbolic_slack_matrix(P)
    Bi,Tri = binomials_and_trinomials_minors(S, d+2) 
    B = coefficients_binomials(Bi, numvar)
    L = list(B)
    T = coefficients_trinomials(Tri, numvar)
    for element in T:
        L.append(element[0])
    A = Matrix(L)
    H = A.hermite_form(include_zero_rows=False)
    for element in T:
        B = Matrix(L + [element[1]])
        if B.hermite_form(include_zero_rows=False) == H:
            return 'The polytope is not complex psd-minimal'
    return 'It cannot be concluded that the polytope is not complex psd-minimal'

# Given a d-polytope P = [F1,...,Fm] as a list of facets where Fi is a list with the vertices in that facet,
# it returns if we can conclude that the polytope is not complex psd-minimal using the second trinomial obstruction
# lattice criterion with the (d+2)-binomials and trinomials minors of the symbolic slack matrix of P  
def ComplexPsdMinimality2(P, d):
    numvar = num_variables(P)
    S = symbolic_slack_matrix(P)
    Bi,Tri = binomials_and_trinomials_minors(S, d+2) 
    B = coefficients_binomials(Bi, numvar)
    L = list(B)
    T = coefficients_trinomials(Tri, numvar)
    for element1 in T:
        for element2 in T:
            L.append(element1[0]+element2[0])
    A = Matrix(L)
    H = A.hermite_form(include_zero_rows=False)
    for element in T:
        B = Matrix(L + [2*element[1]])
        if B.hermite_form(include_zero_rows=False) == H:
            return 'The polytope is not complex psd-minimal'
    return 'It cannot be concluded that the polytope is not complex psd-minimal'

# Given a 3-polytope P = [F1,...,Fm] as a list of facets where Fi is a list with the vertices in that facet,
# it returns if we can conclude that the polytope is not complex psd-minimal using the first trinomial obstruction
# lattice criterion with the (d+2)-binomials and trinomials minors coming from facets and vertices of degree 4 
def ComplexPsdMinimality1_3d(P):
    numvar = num_variables(P)
    S = symbolic_slack_matrix(P) 
    B = coefficients_binomials(binomials_3d(S), numvar)
    L = list(B)
    T = coefficients_trinomials(trinomials_3d(S), numvar)
    for element in T:
        L.append(element[0])
    A = Matrix(L)
    H = A.hermite_form(include_zero_rows=False)
    for element in T:
        B = Matrix(L + [element[1]])
        if B.hermite_form(include_zero_rows=False) == H:
            return 'The polytope is not complex psd-minimal'
    return 'It cannot be concluded that the polytope is not complex psd-minimal'

# Given a 3-polytope P = [F1,...,Fm] as a list of facets where Fi is a list with the vertices in that facet,
# it returns if we can conclude that the polytope is not complex psd-minimal using the second trinomial obstruction
# lattice criterion with the (d+2)-binomials and trinomials minors coming from facets and vertices of degree 4 
def ComplexPsdMinimality2_3d(P):
    numvar = num_variables(P)
    S = symbolic_slack_matrix(P) 
    B = coefficients_binomials(binomials_3d(S), numvar)
    L = list(B)
    T = coefficients_trinomials(trinomials_3d(S), numvar)
    for element1 in T:
        for element2 in T:
            L.append(element1[0]+element2[0])
    A = Matrix(L)
    H = A.hermite_form(include_zero_rows=False)
    for element in T:
        B = Matrix(L + [2*element[1]])
        if B.hermite_form(include_zero_rows=False) == H:
            return 'The polytope is not complex psd-minimal'
    return 'It cannot be concluded that the polytope is not complex psd-minimal'

In [18]:
# Example (this is a 3-polytope)
P = [[1,2,3,4],[1,2,5,6],[1,4,5],[4,5,6],[3,4,6,7],[3,7,8],[2,6,7,8],[2,3,8]]
print ComplexPsdMinimality1(P, 3)
print ComplexPsdMinimality2(P, 3)
print ComplexPsdMinimality1_3d(P)
print ComplexPsdMinimality2_3d(P)

It cannot be concluded that the polytope is not complex psd-minimal
The polytope is not complex psd-minimal
It cannot be concluded that the polytope is not complex psd-minimal
The polytope is not complex psd-minimal
